In [5]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
from gensim.models import word2vec

In [13]:
model=word2vec.Word2Vec.load("/Users/lwd011204/书籍爬虫/book_spyder/resources/word2vec.model")
model.vector_size,len(model.wv)

(100, 17174)

In [15]:
class config:
    num_class=15
    dropout_rate=0.1
    max_text_length=46
    vocab_size=17146
    vec_dim=100
    kernel_size=[2,3,4]
    feature_size=100#一维卷积卷积核个数
    weight=torch.tensor(model.wv.vectors)
    vocab_size=len(model.wv)


In [10]:
config.weight

tensor([[-1.5960e-03,  2.6173e-03,  5.0790e-03,  ..., -1.1207e-02,
          6.5536e-05,  9.2603e-03],
        [-9.4142e-03,  5.8541e-03,  5.6860e-03,  ..., -4.2937e-03,
         -8.8872e-03,  4.7687e-03],
        [-3.6117e-04,  4.5825e-03, -6.3246e-03,  ..., -7.1254e-04,
          8.4597e-03, -6.5693e-03],
        ...,
        [-8.7692e-03,  4.4757e-03, -3.3578e-03,  ...,  3.6505e-03,
         -1.0899e-03, -5.5242e-04],
        [-5.8711e-03,  2.7486e-04,  8.7491e-03,  ...,  5.2116e-03,
         -7.4819e-03, -3.7213e-03],
        [-4.8564e-03,  8.8754e-03,  1.4660e-03,  ..., -8.6117e-03,
         -5.8143e-04, -5.4695e-03]])

In [18]:
class TextCNN(nn.Module):
    def __init__(self,config) -> None:
        super(TextCNN,self).__init__()
        self.dropout_rate=config.dropout_rate
        self.num_class=config.num_class
        self.config=config
        self.embedding=nn.Embedding(config.vocab_size,config.vec_dim)
        self.convs=nn.ModuleList([nn.Sequential(nn.Conv1d(in_channels=config.vec_dim,out_channels=config.feature_size,kernel_size=h),
        nn.ReLU(),nn.MaxPool1d(kernel_size=config.max_text_length-h+1)) 
        for h in config.kernel_sizes])
        self.fc=nn.Linear(config.feature_size*len(config.kernel_sizes),out_features=config.num_class)
        self.init_weight()
    def init_weight(self,config):
        self.embedding.weight.data.copy(config.weight)
    def forward(self,x):
        embed_x=self.embedding(x)
        embed_x=embed_x.permute(0,2,1)
        out=[conv(embed_x) for conv in self.convs]
        out=torch.cat(out,dim=1)
        out=out.view(-1,out.size(1))
        out=F.dropout(out,p=self.dropout_rate)
        out=self.fc(out)
        return out

In [20]:
net=TextCNN(config)

AttributeError: 'config' object has no attribute 'kernel_sizes'